## Загрузка данных

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import ast
from collections import defaultdict

import phik
from optuna_integration import OptunaSearchCV
# import shap
from ipywidgets import IntProgress
from IPython.display import display
import time

from sklearn.metrics import mean_absolute_percentage_error, make_scorer

# Импорт для разбивки и кодирования данных
from sklearn.impute import SimpleImputer
from sklearn.model_selection import (
    # Разбиение данных
    train_test_split,

    # Поиск гиперпараметров
    RandomizedSearchCV
)
from sklearn.preprocessing import (
    # Кодирование числовых значений
    StandardScaler,
    MinMaxScaler,
    RobustScaler,

    # Кодирование категориальных значение
    OneHotEncoder,
    LabelEncoder
)

# Импорт для создания пайплайна
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Импорт бейслайн моделей
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso
)

# Импорт бустинг моделей
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from optuna.integration import OptunaSearchCV
from optuna.distributions import (
    IntDistribution,
    FloatDistribution,
    CategoricalDistribution
)

from transformers import AutoTokenizer, AutoModel
import torch

RANDOM_STATE = 42

## Создание пайплайна

Создадим пайплайн для обучения моделей

In [ ]:
# # Определяем числовые и категориальные столбцы
# num_col_names = []
# cat_col_names = []
# label_col_names = []
#
# # Пайплайн для числовых значений
# num_pipeline = Pipeline([
#     (
#         'ImputerBeforeScaler',
#         SimpleImputer(strategy='most_frequent', missing_values=np.nan)
#     ),
#     (
#         'num',
#         StandardScaler()
#     )
# ])
#
# # Пайплайн для категориальных значений
# ohe_pipeline = Pipeline([
#     (
#         'OneHotEncoder',
#         OneHotEncoder(drop='first', handle_unknown='ignore', sparse_output=False)
#     )
# ])
#
# # Собираем в один пайплайн по подготовке данных
# data_preprocessor = ColumnTransformer(
#     [
#         ('num', num_pipeline, num_col_names),
#         ('ohe', ohe_pipeline, cat_col_names),
#     ],
#     remainder='passthrough',
# )
#
# # Собираем все в конечный пайплайн
# final_pipeline = Pipeline(
#     [
#         ('preprocessor', data_preprocessor),
#         ('model', DummyClassifier(strategy='median')),
#     ]
# )

## Обучение CatBoost

In [ ]:
# Расширенная сетка гиперпараметров
catboost_params = {
    # Основные параметры
    'iterations': IntDistribution(50, 1000, step=50),  # Увеличенный диапазон
    'learning_rate': FloatDistribution(0.001, 0.3, log=True),  # Более широкий диапазон
    'depth': IntDistribution(4, 12),  # Расширенная глубина деревьев

    # Регуляризация
    'l2_leaf_reg': FloatDistribution(1e-8, 100, log=True),  # Более широкий диапазон
    'random_strength': FloatDistribution(1e-8, 10, log=True),  # Сила случайности

    # Управление бэггингом
    'bagging_temperature': FloatDistribution(0.0, 1.0),  # Температура бэггинга
    'subsample': FloatDistribution(0.5, 1.0),  # Аналогично subsample в LightGBM
    'colsample_bylevel': FloatDistribution(0.5, 1.0),  # Аналог colsample_bytree

    # Дополнительные параметры
    'min_data_in_leaf': IntDistribution(1, 20),  # Минимальное количество объектов в листе
    'max_bin': IntDistribution(64, 512),  # Количество бинов для числовых признаков
    'one_hot_max_size': IntDistribution(2, 16),  # Размер для one-hot кодирования категориальных признаков

    # Стратегии роста деревьев
    'grow_policy': CategoricalDistribution(['SymmetricTree', 'Depthwise', 'Lossguide']),

    # Параметры оценки листьев
    'leaf_estimation_iterations': IntDistribution(1, 10),  # Итерации для оценки листьев
    'leaf_estimation_method': CategoricalDistribution(['Newton', 'Gradient']),  # Метод оптимизации

    # Автоматическая балансировка (для классификации, но можно тестировать)
    # 'auto_class_weights': CategoricalDistribution(['None', 'Balanced', 'SqrtBalanced']),

    # Управление категориальными признаками
    'ctr_leaf_count_limit': IntDistribution(10, 1000),  # Ограничение на количество категорий
    'has_time': CategoricalDistribution([True, False]),  # Учет временного фактора

    # Ранняя остановка
    'early_stopping_rounds': IntDistribution(10, 100)  # Добавлена ранняя остановка
}

In [ ]:
# Используем OptunaSearchCV
catboost_optuna = OptunaSearchCV(
    # estimator=CatBoostRegressor(),
    estimator=CatBoostRegressor(
        random_state=RANDOM_STATE,
        verbose=False,  # Отключаем вывод логов CatBoost
        thread_count=-1,  # Используем все доступные ядра
    ),
    param_distributions=catboost_params,
    scoring=mape_custom_score,  # Метрика для оптимизации
    n_trials=15,  # Количество испытаний
    cv=5,  # Кросс-валидация
    n_jobs=-1,  # Использование всех ядер
    verbose=0,
)

In [ ]:
# Запускаем поиск гиперпараметров
catboost_optuna.fit(X_train, y_train, cat_features=catboost_col_names)

In [ ]:
# Выводим лучшую оценку
catboost_optuna.best_score_

In [ ]:
# Предсказываем таргет
predictions['catboost_pred'] = catboost_optuna.predict(test_df_processed)

## Обучение LightGBM

In [64]:
# Расширенная сетка гиперпараметров для LightGBM
lgbm_params = {
    # Основные параметры
    'num_leaves': IntDistribution(20, 400),  # Количество листьев в дереве
    'max_depth': IntDistribution(3, 10),  # Максимальная глубина деревьев
    'min_child_samples': IntDistribution(5, 200),  # Мин. количество объектов в листе
    'min_child_weight': FloatDistribution(1e-3, 10),  # Вес для ограничения разделения узла

    # Скорость обучения
    'learning_rate': FloatDistribution(0.005, 0.3, log=True),  # Широкий диапазон

    # Регуляризация
    'lambda_l1': FloatDistribution(1e-5, 10, log=True),  # L1-регуляризация
    'lambda_l2': FloatDistribution(1e-5, 10, log=True),  # L2-регуляризация
    'min_gain_to_split': FloatDistribution(1e-3, 10),  # Мин. улучшение для разделения

    # Подвыборка
    'subsample': FloatDistribution(0.4, 1.0),  # Доля данных для обучения
    'subsample_freq': IntDistribution(0, 5),  # Частота бэггинга
    'colsample_bytree': FloatDistribution(0.4, 1.0),  # Доля признаков на дерево

    # Число деревьев
    'n_estimators': IntDistribution(50, 1000),  # Количество итераций

    # Дополнительные параметры
    'max_bin': IntDistribution(64, 1024),  # Количество бинов для числовых признаков
    'cat_smooth': FloatDistribution(1.0, 100.0),  # Гладкость для категориальных признаков
    'cat_l2': FloatDistribution(1e-8, 10),  # L2-регуляризация для категориальных

    # Стратегии построения деревьев
    'boosting_type': CategoricalDistribution(['gbdt', 'dart', 'goss']),  # Тип бустинга

    # Другие параметры
    # 'early_stopping_rounds': IntDistribution(10, 50),  # Ранняя остановка
    'feature_fraction_bynode': FloatDistribution(0.1, 1.0)  # Доля признаков на узел
}

In [65]:
# Настройка Optuna
lgbm_optuna = OptunaSearchCV(
    estimator=LGBMRegressor(
        objective='regression',
        metric='mape',
        boosting_type='gbdt',
        random_state=RANDOM_STATE,
        verbosity=-1,
    ),
    param_distributions=lgbm_params,
    scoring=mape_custom_score,  # Для регрессии
    n_trials=5,  # Количество испытаний
    cv=5,  # Кросс-валидация
    n_jobs=-1,  # Параллельные вычисления
    verbose=0
)

/var/folders/d5/3r484lmx4rx7jdlt6zl0_bkc0000gn/T/ipykernel_39479/2523775932.py:2: ExperimentalWarning: OptunaSearchCV is experimental (supported from v0.17.0). The interface can change in the future.
  lgbm_optuna = OptunaSearchCV(


In [66]:
# Обучаем модель
lgbm_optuna.fit(X_train, y_train)

[I 2025-05-08 17:42:37,993] A new study created in memory with name: no-name-4f7d7c96-516c-42ee-9560-e6e064be731b
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
[LightGBM] [Fatal] Cannot use bagging in GOSS
/Users/moncervers/PycharmProjects/kaggle/hotel_review_score_prediction/.venv/lib/python3.12/site-packages/optuna_integration/sklearn/sklearn.py:394: RuntimeWarning: Mean of empty slice
  trial.set_user_attr("mean_{}".format(name), np.nanmean(array))
/Users/moncervers/PycharmProjects/kaggle/hotel_review_score_prediction/.venv/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:2019: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
[W 2025-05-08 17:43:30,136] Trial 3 failed with parameters: {'num_leaves': 142, 'max_depth': 3, 'min_child_samples': 92, 'min_child_weight': 9.876105

OptunaSearchCV(cv=5,
               estimator=LGBMRegressor(metric='mape', objective='regression',
                                       random_state=42, verbosity=-1),
               n_jobs=-1, n_trials=5,
               param_distributions={'boosting_type': CategoricalDistribution(choices=('gbdt', 'dart', 'goss')),
                                    'cat_l2': FloatDistribution(high=10.0, log=False, low=1e-08, step=None),
                                    'cat_smooth': FloatDistribution(high=100.0, log=False, low=1...
                                    'min_gain_to_split': FloatDistribution(high=10.0, log=False, low=0.001, step=None),
                                    'n_estimators': IntDistribution(high=1000, log=False, low=50, step=1),
                                    'num_leaves': IntDistribution(high=400, log=False, low=20, step=1),
                                    'subsample': FloatDistribution(high=1.0, log=False, low=0.4, step=None),
                                    'subsample_freq': IntDistribution(high=5, log=False, low=0, step=1)},
               scoring=make_scorer(mape, greater_is_better=False, response_method='predict'))

In [67]:
# Выводим лучшую оценку
lgbm_optuna.best_score_

-11.209251093573007

In [ ]:
# Предсказываем таргет
predictions['lgbm_pred'] = lgbm_optuna.predict(test_df_processed)

## Сохранение submission

In [ ]:
predictions

In [ ]:
submission = predictions[['id', 'lgbm_pred']]
submission

In [ ]:
submission.to_csv('submission.csv', index=False)